# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [1]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [710]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1.
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1.
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    return x_tilde
end

function starting_points(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(-ρ*ones(n_cols),spzeros(n_rows))

    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :L))
    init_xλ = J_fact \ [c ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = 0., 0.
    else
        δx_l1 = max(-1.5*minimum(x0_l1 - lvar[ilow]), 0.)
        δs_l1 = max(-1.5*minimum(s0_l[ilow]), 0.)
    end

    if length(iupp) == 0
        δx_u1, δs_u1 = 0., 0.
    else
        δx_u1 = max(-1.5*minimum((uvar[iupp] - x0_u1)), 0.)
        δs_u1 = max(-1.5*minimum(s0_u[iupp]), 0.)
    end
    
    x0_l1 .+= δx_l1
    x0_u1 .-= δx_u1
    s0_l1 = s0_l[ilow] .+ δs_l1
    s0_u1 = s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = s0_l1'*(x0_l1 - lvar[ilow]), s0_u1'*(uvar[iupp] - x0_u1)
    if length(ilow) == 0
        δx_l2, δs_l2 = 0., 0.
    else
        δx_l2 = δx_l1 + 0.5 *  xs_l1 / sum(s0_l1)
        δs_l2 = δs_l1 + 0.5 * xs_l1 / sum(x0_l1-lvar[ilow])
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = 0., 0.
    else
        δx_u2 = δx_u1 + 0.5 *  xs_u1 / sum(s0_u1)
        δs_u2 = δs_u1 + 0.5 * xs_u1 / sum(uvar[iupp]-x0_u1)
    end
    println(δx_l1)
    x0[ilow] .+= δx_l2 
    x0[iupp] .-= δx_u2
    s0_l[ilow] = s0_l[ilow] .+ max(δs_l2, δs_u2)
    s0_u[iupp] = s0_u[iupp] .+ max(δs_l2, δs_u2)

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end



function starting_points2(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(-ones(n_cols),spzeros(n_rows))

    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :L))
    init_xλ = J_fact \ [zeros(n_cols) ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]

    for i in ilow
        δx_l = max(-1.5*(x0[i] - lvar[i]), 1.)
        δs_l = max(-1.5*s0_l[i], 1e-6)
        x0[i] += δx_l 
        s0_l[i] += δs_l 
    end

    for i in iupp
        δx_u = max(-1.5*(uvar[i] - x0[i]), 1.)
        δs_u = max(-1.5*s0_u[i], 1.e-6)
        x0[i] -= δx_u 
        s0_u[i] += δs_u 
    end

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end


function starting_points3(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(-ones(n_cols),spzeros(n_rows))

    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :L))
    init_xλ = J_fact \ [zeros(n_cols) ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = 0., 0.
    else
        δx_l1 = max(-1.5*minimum(x0_l1 - lvar[ilow]), 1.)
        δs_l1 = max(-1.5*minimum(s0_l[ilow]), 1.e-6)
    end

    if length(iupp) == 0
        δx_u1, δs_u1 = 0., 0.
    else
        δx_u1 = max(-1.5*minimum((uvar[iupp] - x0_u1)), 1.)
        δs_u1 = max(-1.5*minimum(s0_u[iupp]), 1.e-6)
    end
    
    x0_l1 .+= δx_l1
    x0_u1 .-= δx_u1
    s0_l1 = s0_l[ilow] .+ δs_l1
    s0_u1 = s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = s0_l1'*(x0_l1 - lvar[ilow]), s0_u1'*(uvar[iupp] - x0_u1)
    if length(ilow) == 0
        δx_l2, δs_l2 = 0., 0.
    else
        δx_l2 = δx_l1 + 0.5 *  xs_l1 / sum(s0_l1)
        δs_l2 = δs_l1 + 0.5 * xs_l1 / sum(x0_l1-lvar[ilow])
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = 0., 0.
    else
        δx_u2 = δx_u1 + 0.5 *  xs_u1 / sum(s0_u1)
        δs_u2 = δs_u1 + 0.5 * xs_u1 / sum(uvar[iupp]-x0_u1)
    end
    x0[ilow] .+= δx_l2 
    x0[iupp] .-= δx_u2
    s0_l[ilow] = s0_l[ilow] .+ max(δs_l2, δs_u2)
    s0_u[iupp] = s0_u[iupp] .+ max(δs_l2, δs_u2)

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end

function starting_points4(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    x0 = A\b
    
    if length(ilow) == 0
        δx_l1 = 0.
    else
        δx_l1 = max(-1.5*minimum(x0[ilow] - lvar[ilow]), 1.)
    end

    if length(iupp) == 0
        δx_u1 = 0.
    else
        δx_u1 = max(-1.5*minimum((uvar[iupp] - x0[iupp])), 1.)
    end
    
    x0[ilow] .+= δx_l1
    x0[iupp] .-= δx_u1
    
    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end
    
    @assert all(x0 .> lvar) && all(x0 .< uvar)
    λ0 = sparse(A') \ (c+Q*x0)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = dual_val[iupp]
    
    s0_l[ilow] .= 1.
    s0_u[iupp] .= 1.    
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end

starting_points4 (generic function with 1 method)

In [711]:
function compute_α_dual(v, dir_v)
    n = length(v)
    if n == 0
        return 1.
    end
    α = 1.
    for i=1:n
        if dir_v[i] < 0.
            α_new = -v[i] * 0.999 / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    α_l, α_u = 1., 1.
    for i=1:n
        if dir_v[i] > 0.
            α_u_new = (uvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < 0.
            α_l_new = (lvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end

function compute_μ(x_l, x_u, s_l, s_u, lvar, uvar, nb_low, nb_upp)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s_l' * (x_l-lvar) + s_u' * (uvar-x_u)) / (nb_low + nb_upp)
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [712]:
function solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                    iupp, lvar, uvar, n_cols)
    
    F_x_λ_aff = [-rc 
                 -rb]
    F_x_λ_aff[ilow] += s_l[ilow]
    F_x_λ_aff[iupp] -= s_u[iupp]
    
    Δ_x_λ_aff = J_fact\F_x_λ_aff
    Δ_aff = [Δ_x_λ_aff
             -s_l[ilow] - s_l[ilow].*Δ_x_λ_aff[1:n_cols][ilow]./(x[ilow]-lvar[ilow])
             - s_u[iupp] - s_u[iupp].*Δ_x_λ_aff[1:n_cols][iupp]./(x[iupp]-uvar[iupp])]
    return Δ_aff
end

function solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                   Δ_aff, n_cols, n_rows, n_low)


    rxs_l = -σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
    rxs_u = σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]

    F_x_λ_cc = zeros(n_cols+n_rows,1)
    F_x_λ_cc[ilow] += rxs_l./(x[ilow]-lvar[ilow])
    F_x_λ_cc[iupp] -= rxs_u./(x[iupp]-uvar[iupp])

#     F_x_λ_cc = [ I_low*inv_X_L*rxs_l + I_op_upp*inv_X_U*rxs_u
#                                    Or                 ]
    Δ_x_λ_cc = J_fact\F_x_λ_cc
    Δ_cc = [Δ_x_λ_cc
            -(rxs_l+s_l[ilow].*Δ_x_λ_cc[1:n_cols][ilow])./(x[ilow]-lvar[ilow])
            -(rxs_u+s_u[iupp].*Δ_x_λ_cc[1:n_cols][iupp])./(x[iupp]-uvar[iupp]) ]
    return Δ_cc
end


solve_augmented_system_cc (generic function with 1 method)

In [732]:
function mehrotraPCQuadBounds(QM; max_iter=300, ϵ_pdd=1e-8, ϵ_rb=1e-6, ϵ_rc=1e-6,
                              tol_Δx=1e-16, ϵ_μ=0., max_time=60., display=true)
    
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp = [QM.meta.ilow; QM.meta.irng], [QM.meta.iupp; QM.meta.irng] # finite bounds index
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    A = jac(QM, Oc)
    Arows, Acols, Avals = findnz(A)
    n_rows, n_cols = size(A) 
    Q = hess(QM, Oc)  # lower triangular
    Qrows, Qcols, Qvals = findnz(Q)
    c = grad(QM, Oc)
    c0 = obj(QM, Oc)
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    
    # init regularization values
    ρ, δ = 1.e-8, 1.e-6

    J_augmrows = vcat(Qrows, Arows.+n_cols, n_cols+1:n_cols+n_rows, 1:n_cols)
    J_augmcols = vcat(Qcols, Acols, n_cols+1:n_cols+n_rows, 1:n_cols)
    tmp_diag = zeros(n_cols)
    J_augmvals = vcat(-Qvals, Avals, δ*ones(n_rows), tmp_diag)
    J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
    
    x, λ, s_l, s_u = starting_points3(Q, A, b, c, lvar, uvar, ilow, iupp, 
                                     QM.meta.irng, J_augm, 1e-6, n_rows, n_cols)
    
#     x = init_x0_lsq(A, b, lvar, uvar)
#     @assert all(x .> lvar) && all(x .< uvar)
#     s_l, s_u = copy(Oc), copy(Oc)
#     s_l[ilow] .= 1.
#     s_u[iupp] .= 1.
    Qx = Q * x
#     λ = sparse(A') \ (c+Qx) # least square initialisation, s_0 = stilde_0
    
    rb = A * x - b
    rc = -Qx + A' * λ + s_l - s_u - c
    μ = compute_μ(x[ilow], x[iupp], 
                  s_l[ilow], s_u[iupp], 
                  lvar[ilow], uvar[iupp],
                  n_low, n_upp)

    k = 0

    # stopping criterion
    xTQx_2 = x' * Qx / 2.
    cTx = c' * x
    pri_obj = xTQx_2 + cTx + c0
    dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0
    pdd = abs(pri_obj - dual_obj ) / (1. + abs(pri_obj)) 
    max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
    optimal = pdd < ϵ_pdd && max_rb < ϵ_rb && max_rc < ϵ_rc
    
    n_Δx = 0.
    small_Δx, small_μ = false, μ < ϵ_μ
    Δt = time() - start_time
    tired = Δt > max_time

    # display
    if display == true
        @info log_header([:k, :pri_obj, :pdd, :max_rb, :max_rc, :n_Δx, :μ],
                         [Int, Float64, Float64, Float64, Float64, Float64, Float64, Float64],
                        hdr_override=Dict(:k=>"Iter", :pri_obj=>"primal", :pdd=>"pdd",
                                          :max_rb=>"rb cond", :max_rc=>"rc cond",
                                          :n_Δx=>"‖Δx‖", :μ=>"μ"))
        @info log_row([k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
    end

    while k<max_iter && !optimal && !tired # && !small_μ && !small_μ
        
            
            # Affine scaling direction
        tmp_diag .= -ρ
        tmp_diag[ilow] -= s_l[ilow] ./ (x[ilow] - lvar[ilow]) 
        tmp_diag[iupp] -= s_u[iupp] ./ (uvar[iupp] - x[iupp]) 
        J_augmvals[end-n_cols+1:end] = tmp_diag

        J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
    
        J_fact = ldlt(Symmetric(J_augm, :L))
        #J_fact = lu(J_augm, check=true)
        
        Δ_aff = solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                           iupp, lvar, uvar, n_cols)

        α_aff_pri = compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)
        
        μ_aff = compute_μ(x[ilow] + α_aff_pri * Δ_aff[1:n_cols][ilow],
                          x[iupp] + α_aff_pri * Δ_aff[1:n_cols][iupp],
                          s_l[ilow] + α_aff_dual_final * Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low],
                          s_u[iupp] + α_aff_dual_final * Δ_aff[n_rows+n_cols+n_low+1: end],
                          lvar[ilow], uvar[iupp],
                          n_low, n_upp)
        
        σ = (μ_aff / μ)^3

        # corrector and centering step
        
        Δ_cc = solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                         Δ_aff, n_cols, n_rows, n_low)


        Δ = Δ_aff + Δ_cc # final direction
        
        α_pri = compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        # new parameters
        x += α_pri * Δ[1:n_cols]
        λ += α_dual_final * Δ[n_cols+1: n_rows+n_cols]
        s_l[ilow] += α_dual_final * Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u[iupp] += α_dual_final * Δ[n_rows+n_cols+n_low+1: end]
        
        n_Δx = α_pri * norm(Δ[1:n_cols])
        μ = compute_μ(x[ilow], x[iupp], 
                      s_l[ilow], s_u[iupp], 
                      lvar[ilow], uvar[iupp],
                      n_low, n_upp)
        
        Qx = Q * x
        xTQx_2 = x' * Qx / 2.
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0

        rb = A * x - b
        rc = -Qx + A' * λ + s_l - s_u - c
        
        # update stopping criterion values:
        
        pdd = abs(pri_obj - dual_obj ) / (1. + abs(pri_obj)) 
        max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
        optimal = pdd < ϵ_pdd && max_rb < ϵ_rb && max_rc < ϵ_rc
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1
        
                    ### case ρ=δ and ρ /= 10 each iter
#         if ρ >= 1e-7
#             for j in n_cols+1:n_rows+n_cols
#                 J_augm[j,j] /= 10.
#             end
#             ρ /= 10.
#         end
        Δt = time() - start_time
        tired = Δt > max_time
        
        if display == true
            @info log_row([k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
        end
        
        
    end
    
    elapsed_time = time() - start_time
    
    if k>= max_iter
        status = :max_iter
    elseif tired
        status = :max_time 
    elseif optimal
        status = :acceptable
    else
        status = :unknown
    end
    
    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj , 
                                  dual_feas = max_rc, 
                                  primal_feas = max_rb,
                                  multipliers = λ,
                                  multipliers_L = s_l,
                                  multipliers_U = s_u,
                                  iter = k, elapsed_time=elapsed_time)
    return stats
end



mehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [733]:
# probleme1
Q = [6. 2 1
     2 5 2
     1 2 4]
c = [-8.; -3; -3]
c0 = 0.
A = [1. 0 1
    0 1 1]
b = [0.; 3]
lvar = [0.;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [734]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [735]:
stats_mpc1 =  mehrotraPCQuadBounds(SM)
println(stats_mpc1)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.499999999873374
  primal feasibility: 8.690825836765725e-12
  dual feasibility: 1.0575746056673552e-8
  solution: [2.3382226703417235e-13  2.9999999999895453  1.763967867108076e-12]
  multipliers: [-20.931294911378433  11.99999998936386]
  multipliers_L: [12.931294911852033  8.58733956979702e-12  11.931294921995049]
  multipliers_U: [0.0  0.0  0.0]
  iterations: 5
  elapsed time: 0.0


┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[732]:68
┌ Info:  0.0e+00   1.1e+02   2.2e+00   6.5e+00   1.0e+01   0.0e+00   9.5e+01
└ @ Main In[732]:73
┌ Info:  1.0e+00   1.5e+01   2.7e+00   1.5e-01   9.4e+00   5.2e+00   5.3e+00
└ @ Main In[732]:163
┌ Info:  2.0e+00   1.4e+01   6.0e-03   1.9e-03   3.1e-01   2.1e-01   1.6e-02
└ @ Main In[732]:163
┌ Info:  3.0e+00   1.4e+01   7.3e-04   2.0e-06   3.5e-03   1.8e-03   1.7e-05
└ @ Main In[732]:163
┌ Info:  4.0e+00   1.3e+01   1.5e-06   2.0e-09   7.1e-06   1.8e-06   1.7e-08
└ @ Main In[732]:163
┌ Info:  5.0e+00   1.3e+01   2.2e-09   8.7e-12   1.1e-08   3.8e-09   1.7e-11
└ @ Main In[732]:163


In [736]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
stats_mpc_verif =  mehrotraPCQuadBounds(SM_verif);
println(stats_mpc_verif)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.499999999873374
  primal feasibility: 8.690825836765725e-12
  dual feasibility: 1.0575746056673552e-8
  solution: [-2.3382226703417235e-13  -2.9999999999895453  -1.763967867108076e-12]
  multipliers: [-20.931294911378433  11.99999998936386]
  multipliers_L: [0.0  0.0  0.0]
  multipliers_U: [12.931294911852033  8.58733956979702e-12  11.931294921995049]
  iterations: 5
  elapsed time: 0.0009999275207519531


┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[732]:68
┌ Info:  0.0e+00   1.1e+02   2.2e+00   6.5e+00   1.0e+01   0.0e+00   9.5e+01
└ @ Main In[732]:73
┌ Info:  1.0e+00   1.5e+01   2.7e+00   1.5e-01   9.4e+00   5.2e+00   5.3e+00
└ @ Main In[732]:163
┌ Info:  2.0e+00   1.4e+01   6.0e-03   1.9e-03   3.1e-01   2.1e-01   1.6e-02
└ @ Main In[732]:163
┌ Info:  3.0e+00   1.4e+01   7.3e-04   2.0e-06   3.5e-03   1.8e-03   1.7e-05
└ @ Main In[732]:163
┌ Info:  4.0e+00   1.3e+01   1.5e-06   2.0e-09   7.1e-06   1.8e-06   1.7e-08
└ @ Main In[732]:163
┌ Info:  5.0e+00   1.3e+01   2.2e-09   8.7e-12   1.1e-08   3.8e-09   1.7e-11
└ @ Main In[732]:163


In [737]:
### pb verif 2
c2 = [1.; 0; 0]
A2 = Matrix([1., 1, 1]')
b2 = [1.];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
stats_mpc_verif2 =  mehrotraPCQuadBounds(SM_verif2);
println(stats_mpc_verif2)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 1.2301937055839214e-11
  primal feasibility: 1.7763568394002505e-15
  dual feasibility: 4.440892098500626e-16
  solution: [1.2301937055839214e-11  0.49999999999384814  0.49999999999384814]
  multipliers: [-4.1629755566536037e-13]
  multipliers_L: [1.000000000000416

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[732]:68
┌ Info:  0.0e+00   2.0e+00   7.8e-01   5.0e+00   3.3e-01   0.0e+00   2.0e+00
└ @ Main In[732]:73
┌ Info:  1.0e+00   1.5e-01   4.9e-01   8.3e-08   1.9e-08   2.9e+00   1.9e-01
└ @ Main In[732]:163
┌ Info:  2.0e+00   1.2e-02   1.3e-02   4.3e-07   1.9e-09   1.7e-01   4.2e-03
└ @ Main In[732]:163
┌ Info:  3.0e+00   1.2e-05   1.3e-05   8.5e-10   1.2e-10   1.5e-02   4.2e-06
└ @ Main In[732]:163
┌ Info:  4.0e+00   1.2e-08   1.3e-08   1.3e-12   2.5e-13   1.5e-05   4.2e-09
└ @ Main In[732]:163
┌ Info:  5.0e+00   1.2e-11   1.3e-11   1.8e-15   4.4e-16   1.5e-08   4.2e-12
└ @ Main In[732]:163


  4.164820986593909e-13  4.164820986593909e-13]
  multipliers_U: [0.0  0.0  0.0]
  iterations: 5
  elapsed time: 0.012000083923339844


# Lecture des donnees .SIF

In [723]:

function createQuadraticModel(qpdata; name="qp_pb")
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, name=name)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

# Benchmark

In [724]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [725]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
SM2.meta

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   



In [726]:
stats_mpc2 =  mehrotraPCQuadBounds(SM2)
println(stats_mpc2)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -464.7531413819036
  primal feasibility: 4.374101081339177e-11
  dual feasibility: 1.5756160820501464e-9
  solution: [79.99999998238128  25.49999987860402  54.50000010376521  84.79999998136691 ⋯ 133.69844681558686]
  multipliers: [-0.6285714348648097  2.1522203405038338e-8  -0.3447714131747257  -0.22857142999013158 ⋯ -9.756448646743413e-10]
  multipliers_L: [7.446471491924801e-10  5.166563598900057e-9  6.71370733600795e-10  7.025384740366368e-10 ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 8.547664680342902e-10]
  iterations: 10
  elapsed time: 0.023000001907348633


┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[717]:68
┌ Info:  0.0e+00   4.5e+02   4.6e+00   1.9e+01   5.9e+00   0.0e+00   3.7e+01
└ @ Main In[717]:73
┌ Info:  1.0e+00   3.6e+02   5.4e+00   1.8e+01   5.7e+00   1.5e+01   3.6e+01
└ @ Main In[717]:163
┌ Info:  2.0e+00  -5.5e+00   2.4e+02   1.6e+01   5.3e+00   6.3e+01   3.6e+01
└ @ Main In[717]:163
┌ Info:  3.0e+00   3.7e+01   3.5e+01   4.0e-06   4.3e+00   2.2e+02   3.3e+01
└ @ Main In[717]:163
┌ Info:  4.0e+00   6.9e+01   1.1e+01   3.1e-06   2.0e+00   4.5e+01   1.8e+01
└ @ Main In[717]:163
┌ Info:  5.0e+00  -1.6e+02   1.7e+00   9.1e-07   1.1e+00   3.1e+02   9.3e+00
└ @ Main In[717]:163
┌ Info:  6.0e+00  -4.0e+02   3.1e-01   3.8e-07   2.5e-01   5.0e+02   4.0e+00
└ @ Main In[717]:163
┌ Info:  7.0e+00  -4.6e+02   4.1e-02   5.1e-07   5.0e-02   1.1e+02   6.9e-01
└ @ Main In[717]:163
┌ Info:  8.0e+00  -4.6e+02   5.5e-03   1.6e-07   1.5e-03   1.6e+01   6.0e-02
└ @ Main In[717]:163
┌ Info:  9.0e+00  -4.6

In [589]:
@benchmark mehrotraPCQuadBounds(SM2,  display=false)

BenchmarkTools.Trial: 
  memory estimate:  1.12 MiB
  allocs estimate:  4311
  --------------
  minimum time:     1.074 ms (0.00% GC)
  median time:      2.638 ms (0.00% GC)
  mean time:        2.818 ms (2.47% GC)
  maximum time:     10.085 ms (36.79% GC)
  --------------
  samples:          1770
  evals/sample:     1

In [727]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
SM3 = SlackModel(QM3)
SM3.meta

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  Problem name: qp_pb-slack
   All variables: ████████████████████ 68     All constraints: ████████████████████ 43    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ██████████████⋅⋅⋅⋅⋅⋅ 47               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 12               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 43    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 43    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 89.30% sparsity)   313   



In [728]:
stats_mpc3 =  mehrotraPCQuadBounds(SM3);
println(stats_mpc3)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[717]:68
┌ Info:  0.0e+00  -1.0e+03   5.9e-01   1.1e+03   6.1e+00   0.0e+00   6.2e+00
└ @ Main In[717]:73
┌ Info:  1.0e+00  -8.1e+02   3.9e-01   8.3e+02   2.7e+00   2.6e+01   7.6e+00
└ @ Main In[717]:163
┌ Info:  2.0e+00  -4.0e+02   2.7e-02   4.1e+02   1.8e+00   4.8e+01   6.3e+00
└ @ Main In[717]:163
┌ Info:  3.0e+00  -2.1e+02   4.3e-01   2.0e+02   1.3e+00   6.6e+01   5.2e+00
└ @ Main In[717]:163
┌ Info:  4.0e+00  -9.9e+01   7.6e-01   4.0e+01   7.4e-01   2.6e+02   3.2e+00
└ @ Main In[717]:163
┌ Info:  5.0e+00  -4.3e+02   6.8e-01   9.4e+00   5.4e-01   1.9e+03   3.2e+00
└ @ Main In[717]:163
┌ Info:  6.0e+00  -8.4e+02   7.7e-01   9.0e+00   4.8e-01   2.1e+03   3.6e+00
└ @ Main In[717]:163
┌ Info:  7.0e+00  -1.2e+03   7.7e-01   8.8e+00   4.6e-01   2.0e+03   4.3e+00
└ @ Main In[717]:163
┌ Info:  8.0e+00  -1.3e+03   7.0e-01   8.1e+00   4.3e-01   6.1e+02   4.9e+00
└ @ Main In[717]:163
┌ Info:  9.0e+00  -1.2

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -1749.9001298835497
  primal feasibility: 1.0548539819410507e-10
  dual feasibility: 2.486694092554803e-11
  solution: [0.8118235580742549  5.052597225226967e-9  4.414248236606294e-9  5.145222497127126e-9 ⋯ -783.4591279889489]
  multipliers: [17.26920818755939  17.214101764746378  16.942045650083447  16.65983849307432 ⋯ -6.189044279462761e-12]
  multipliers_L: [1.0787673432687917e-10  0.06381181585519778  0.0489616997259479  0.10208327529545538 ⋯ 0.0]
  multipliers_U: [0.0  1.3544009478523228e-11  0.0  0.0 ⋯ 1.801352770579165e-13]
  iterations: 19
  elapsed time: 0.05499982833862305


┌ Info:  1.7e+01  -1.7e+03   1.1e-05   5.1e-05   4.1e-07   4.9e+01   2.4e-04
└ @ Main In[717]:163
┌ Info:  1.8e+01  -1.7e+03   1.5e-08   1.1e-07   6.2e-09   8.3e-01   4.0e-07
└ @ Main In[717]:163
┌ Info:  1.9e+01  -1.7e+03   1.3e-11   1.1e-10   2.5e-11   2.6e-03   4.0e-10
└ @ Main In[717]:163


In [331]:
@benchmark mehrotraPCQuadBounds(SM3, display = false)

BenchmarkTools.Trial: 
  memory estimate:  3.24 MiB
  allocs estimate:  7103
  --------------
  minimum time:     2.365 ms (0.00% GC)
  median time:      2.441 ms (0.00% GC)
  mean time:        2.631 ms (4.12% GC)
  maximum time:     5.616 ms (34.05% GC)
  --------------
  samples:          1898
  evals/sample:     1

In [730]:
# problem 4 SC50A  obj  -6.4575077059E+01
pb4 = string(path_pb, "\\BNL2.SIF")
qpdata4 = readqps(pb4)
QM4 = createQuadraticModel(qpdata4)
SM4 = SlackModel(QM4)
SM4.meta

┌ Info: Using 'BNL2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJ' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 11294)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 4486   All constraints: ████████████████████ 2324  
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ██████████████████⋅⋅ 4004             lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 482              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 2324  
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 2324  
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.86% sparsity)   14996 



In [731]:
stats_mpc4 =  mehrotraPCQuadBounds(SM4, max_iter = 150);
println(stats_mpc4)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[717]:68
┌ Info:  0.0e+00   1.5e+06   1.0e+00   1.0e+06   2.0e+01   0.0e+00   1.3e+03
└ @ Main In[717]:73
┌ Info:  1.0e+00   1.3e+06   1.0e+00   1.0e+06   1.9e+01   3.1e+03   1.3e+03
└ @ Main In[717]:163
┌ Info:  2.0e+00   1.3e+06   1.0e+00   9.9e+05   1.8e+01   3.1e+03   1.3e+03
└ @ Main In[717]:163
┌ Info:  3.0e+00   1.2e+06   1.0e+00   8.3e+05   1.3e+01   1.4e+04   1.3e+03
└ @ Main In[717]:163
┌ Info:  4.0e+00   7.4e+05   1.1e+00   5.2e+05   8.9e+00   3.1e+04   8.9e+02
└ @ Main In[717]:163
┌ Info:  5.0e+00   5.0e+05   1.1e+00   3.4e+05   3.9e+00   1.7e+04   6.1e+02
└ @ Main In[717]:163
┌ Info:  6.0e+00   2.8e+05   1.2e+00   1.9e+05   1.8e+00   3.0e+04   3.6e+02
└ @ Main In[717]:163
┌ Info:  7.0e+00   1.4e+05   1.3e+00   8.9e+04   3.4e-01   2.1e+04   1.8e+02
└ @ Main In[717]:163
┌ Info:  8.0e+00   6.2e+04   1.5e+00   3.7e+04   1.1e-01   2.5e+04   8.2e+01
└ @ Main In[717]:163
┌ Info:  9.0e+00   2.2

┌ Info:  8.3e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  8.4e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  8.5e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  8.6e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  8.7e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  8.8e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  8.9e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  9.0e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  9.1e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  9.2e+01       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  9.3e+01    

Generic Execution stats
  status: maximum iteration
  objective value: NaN
  primal feasibility: NaN
  dual feasibility: NaN
  solution: [NaN  NaN  NaN  NaN ⋯ NaN]
  multipliers: [NaN  NaN  NaN  NaN ⋯ NaN]
  multipliers_L: [NaN  NaN  NaN  NaN ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0

┌ Info:  1.5e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  1.5e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163
┌ Info:  1.5e+02       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[717]:163


 ⋯ NaN]
  iterations: 150
  elapsed time: 9.233999967575073


# Tests sur tous les problèmes

In [347]:
df_test = DataFrame(id = Int[],
         name = String[],
         status = Symbol[],
         objective = Float64[],
         time = Float64[],
         iter = Int[],
         primal_feas = Float64[],
         dual_feas = Float64[])



i = 0
i_stop = 4   #number of problems to solve
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   &&# problems with fixed variables
                                        !(file_name in ["25FV47.SIF"; "BNL1.SIF";
                                                        "BRANDY.SIF"; "CRE-A.SIF"]) # singular error
                            
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end

        SM_i = SlackModel(createQuadraticModel(qpdata_i))

        stats_mpc_i =  mehrotraPCQuadBounds(SM_i, display=false)
        
        # 
        push!(df_test, Dict(:id => i,
                             :name => file_name ,
                             :status => stats_mpc_i.status,
                             :objective => stats_mpc_i.objective,
                             :time => stats_mpc_i.elapsed_time,
                             :iter => stats_mpc_i.iter,
                             :primal_feas => stats_mpc_i.primal_feas,
                             :dual_feas => stats_mpc_i.dual_feas))
    end
    
    if i == i_stop
        break
        break
    end
end

markdown_table(stdout, df_test)

ADLITTLE.SIF
AFIRO.SIF


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


AGG.SIF
AGG2.SIF


┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


|    id |         name |     status | objective |     time |  iter | primal_fe ⋯
|-------|--------------|------------|-----------|----------|-------|---------- ⋯
|     1 | ADLITTLE.SIF | acceptable |   2.3e+05 |  4.0e-03 |    14 |     1.6e- ⋯
|     2 |    AFIRO.SIF | acceptable |  -4.6e+02 |  1.0e-03 |    11 |     2.3e- ⋯
|     3 |      AGG.SIF | acceptable |  -3.6e+07 |  3.9e-02 |    23 |     5.9e- ⋯
|     4 |     AGG2.SIF | acceptable |  -2.0e+07 |  9.0e-02 |    22 |     1.2e- ⋯


┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [348]:
T = typeof(SM2.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1

In [349]:
# problems: AGG.SIF
qpdata5 = readqps(string(path_pb, "\\AGG2.SIF"))
QM5 = createQuadraticModel(qpdata5)
SM5 = SlackModel(QM5);

┌ Info: Using 'AGG2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [350]:
stats_mpc_5 =  mehrotraPCQuadBounds(SM5, max_iter=150)
println(stats_mpc_5)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[331]:77
┌ Info:  0.0e+00   8.2e+11   1.0e+00   3.1e+10   7.5e+01   0.0e+00   3.9e+09
└ @ Main In[331]:82
┌ Info:  1.0e+00   4.7e+11   1.0e+00   1.2e+10   5.2e+01   1.2e+09   1.8e+09
└ @ Main In[331]:176
┌ Info:  2.0e+00   8.8e+10   1.0e+00   9.2e+08   1.8e+00   9.1e+08   1.8e+08
└ @ Main In[331]:176
┌ Info:  3.0e+00   1.7e+09   1.3e+00   1.8e+07   2.0e-01   1.4e+08   4.1e+06
└ @ Main In[331]:176
┌ Info:  4.0e+00   9.5e+08   1.4e+00   9.6e+06   6.7e-02   1.3e+06   2.6e+06
└ @ Main In[331]:176
┌ Info:  5.0e+00   3.2e+08   1.5e+00   3.4e+06   9.9e-03   1.1e+06   9.1e+05
└ @ Main In[331]:176
┌ Info:  6.0e+00   8.3e+07   2.5e+00   9.5e+05   2.0e-03   4.6e+05   3.5e+05
└ @ Main In[331]:176
┌ Info:  7.0e+00  -2.3e+06   2.2e+01   1.8e+05   2.1e-03   3.0e+05   7.6e+04
└ @ Main In[331]:176
┌ Info:  8.0e+00  -6.2e+06   7.1e+00   1.5e+05   1.7e-03   1.0e+05   6.6e+04
└ @ Main In[331]:176
┌ Info:  9.0e+00  -1.1

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -2.0239252353329457e7
  primal feasibility: 1.2096837021380511e-8
  dual feasibility: 5.916584200349462e-8
  solution: [15031.233073265694  0.00023923062989877897  0.0002533786452945386  0.0002637189338962065 ⋯ 547999.9999956683]
  multipliers: [-6.89003112829313e-10  -1.3124969391431051e-8  -2.2428008568508976e-9  -3.6347413421657147e-9 ⋯ 69.78151096967024]
  multipliers_L: [9.141074340835225e-10  0.053999997159137544  0.05099999624509133  0.0489999961548662 ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 2.984999995216971]
  iterations: 

┌ Info:  1.2e+01  -1.5e+07   1.5e+00   4.4e+04   5.4e-04   3.4e+04   3.4e+04
└ @ Main In[331]:176
┌ Info:  1.3e+01  -1.7e+07   1.1e+00   1.2e+04   4.4e-04   4.6e+04   2.6e+04
└ @ Main In[331]:176
┌ Info:  1.4e+01  -1.8e+07   3.0e-01   6.1e+03   2.7e-04   3.6e+04   8.1e+03
└ @ Main In[331]:176
┌ Info:  1.5e+01  -2.0e+07   7.2e-02   1.7e+03   4.5e-04   1.0e+05   2.1e+03
└ @ Main In[331]:176
┌ Info:  1.6e+01  -2.0e+07   1.8e-02   7.2e+01   8.3e-04   2.1e+05   4.9e+02
└ @ Main In[331]:176
┌ Info:  1.7e+01  -2.0e+07   3.9e-03   1.3e+01   3.0e-04   8.2e+04   1.1e+02
└ @ Main In[331]:176
┌ Info:  1.8e+01  -2.0e+07   6.0e-04   1.5e+00   2.2e-04   4.1e+04   1.6e+01
└ @ Main In[331]:176
┌ Info:  1.9e+01  -2.0e+07   6.1e-05   3.8e-02   4.8e-05   9.1e+03   1.7e+00
└ @ Main In[331]:176
┌ Info:  2.0e+01  -2.0e+07   9.1e-06   6.2e-04   7.0e-06   1.1e+03   2.5e-01
└ @ Main In[331]:176
┌ Info:  2.1e+01  -2.0e+07   4.6e-07   1.2e-05   2.0e-06   4.5e+02   1.3e-02
└ @ Main In[331]:176
┌ Info:  2.2e+01  -2

22
  elapsed time: 0.11299991607666016


In [351]:
SM5.meta


  Problem name: qp_pb-slack
   All variables: ████████████████████ 758    All constraints: ████████████████████ 516   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 302              lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: █████████████⋅⋅⋅⋅⋅⋅⋅ 456              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 516   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 516   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 98.79% sparsity)   4740  



In [352]:
SM2.meta.iupp

19-element Array{Int64,1}:
 33
 34
 35
 36
 37
 38
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51

In [250]:
problems = []
i_max = 5
i = 1
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   # problems with fixed variables


        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end
        push!(problems, SlackModel(createQuadraticModel(qpdata_i, name=file_name[1:end-4])))
        
        if i==i_max
            break
        end
        i += 1
    end
end


25FV47.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


ADLITTLE.SIF
AFIRO.SIF
AGG.SIF
AGG2.SIF


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.jul

In [251]:
problems[2].meta.name

"ADLITTLE-slack"

In [270]:
problems_stats = solve_problems(mehrotraPCQuadBounds, problems, 
                                colstats=[:name, :status, :elapsed_time, :objective, :dual_feas, :primal_feas])

┌ Info:            Name           status      Time      f(x)      Dual    Primal  
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:67
┌ Info:    25FV47-slack       acceptable   1.0e+00   5.5e+03   4.1e-09   1.5e-11
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:  ADLITTLE-slack       acceptable   8.0e-03   2.3e+05   1.4e-08   1.4e-07
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:     AFIRO-slack       acceptable   4.0e-03  -4.6e+02   1.9e-11   7.3e-12
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:       AGG-slack       acceptable   6.8e-02  -3.6e+07   2.9e-11   1.1e-09
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:      AGG2-slack       acceptable   2.0e-01  -2.0e+07   2.6e-07   9.5e-

,id,name,nvar,ncon,nequ,status,objective,elapsed_time
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64
1,1,25FV47-slack,1876,821,0,acceptable,5501.85,1.043
2,2,ADLITTLE-slack,138,56,0,acceptable,225495.0,0.0079999
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.00399995
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.0680001
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.202


In [271]:
markdown_table(stdout, problems_stats)

|    id |           name |  nvar |  ncon |  nequ |     status | objective | el ⋯
|-------|----------------|-------|-------|-------|------------|-----------|--- ⋯
|     1 |   25FV47-slack |  1876 |   821 |     0 | acceptable |   5.5e+03 |    ⋯
|     2 | ADLITTLE-slack |   138 |    56 |     0 | acceptable |   2.3e+05 |    ⋯
|     3 |    AFIRO-slack |    51 |    27 |     0 | acceptable |  -4.6e+02 |    ⋯
|     4 |      AGG-slack |   615 |   488 |     0 | acceptable |  -3.6e+07 |    ⋯
|     5 |     AGG2-slack |   758 |   516 |     0 | acceptable |  -2.0e+07 |    ⋯


In [298]:
using JLD2

In [211]:
jldopen("mehrotra.jld2", "w") do file
    file["test"] = problems_stats
end

,id,name,nvar,ncon,nequ,status,objective,elapsed_time
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64
1,1,25FV47-slack,1876,821,0,max_iter,NaN,7.158
2,2,ADLITTLE-slack,138,56,0,acceptable,225495.0,0.0110002
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.00399995
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.063
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.234


In [301]:
res_path = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\results\\mehrotra"

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\results\\mehrotra"

In [305]:
file = jldopen(string(res_path, "\\mehrotra_ldlt2.jld2"), "r")
withenv("COLUMNS" => 600, "LINES" => 200) do
    display(file["stats"])
end

close(file)

,id,name,nvar,ncon,nequ,status,objective,elapsed_time,iter,dual_feas,primal_feas,neval_obj,neval_grad,neval_cons,neval_jcon,neval_jgrad,neval_jac,neval_jprod,neval_jtprod,neval_hess,neval_hprod,neval_jhprod,neval_residual,neval_jac_residual,neval_jprod_residual,neval_jtprod_residual,neval_hess_residual,neval_jhess_residual,neval_hprod_residual,extrainfo
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String
1,1,25FV47-slack,1876,821,0,acceptable,5501.85,0.4496,27,4.05139e-9,1.4525e-11,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
2,2,ADLITTLE-slack,138,56,0,acceptable,225495.0,0.00619888,12,1.38742e-8,1.35373e-7,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.00164199,8,1.92499e-11,7.27596e-12,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.0816481,23,2.91038e-11,1.10592e-9,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.117842,21,2.61011e-7,9.46955e-10,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
6,6,AGG3-slack,758,516,0,acceptable,1.03121e7,0.115678,20,4.17207e-8,2.32831e-10,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
7,7,BANDM,472,305,0,acceptable,-158.628,0.115159,18,1.27826e-10,1.84042e-10,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
8,8,BEACONFD-slack,295,173,0,acceptable,33592.5,0.022325,11,7.6848e-10,4.96003e-10,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
9,9,BLEND-slack,114,74,0,acceptable,-30.8121,0.00574088,10,8.49009e-13,2.09877e-9,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,


In [307]:
file2 = jldopen(string(res_path, "\\mehrotra_ldlt2_no_c.jld2"), "r")
withenv("COLUMNS" => 600, "LINES" => 200) do
    display(file2["stats"])
end

close(file2)

,id,name,nvar,ncon,nequ,status,objective,elapsed_time,iter,dual_feas,primal_feas,neval_obj,neval_grad,neval_cons,neval_jcon,neval_jgrad,neval_jac,neval_jprod,neval_jtprod,neval_hess,neval_hprod,neval_jhprod,neval_residual,neval_jac_residual,neval_jprod_residual,neval_jtprod_residual,neval_hess_residual,neval_jhess_residual,neval_hprod_residual,extrainfo
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String
1,1,25FV47-slack,1876,821,0,acceptable,5501.85,0.483428,27,3.78301e-9,1.93648e-11,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
2,2,ADLITTLE-slack,138,56,0,acceptable,225495.0,0.00696397,14,1.23533e-10,9.48946e-10,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.001513,8,4.20102e-11,1.47793e-11,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.0462821,19,4.87325e-9,2.73079e-7,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.113496,21,4.39767e-7,1.71985e-9,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
6,6,AGG3-slack,758,516,0,acceptable,1.03121e7,0.129101,20,2.65654e-7,3.49246e-10,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
7,7,BANDM,472,305,0,acceptable,-158.628,0.113604,18,7.70026e-12,3.93389e-11,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
8,8,BEACONFD-slack,295,173,0,acceptable,33592.5,0.021965,11,1.32217e-8,1.18282e-8,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
9,9,BLEND-slack,114,74,0,acceptable,-30.8121,0.00695586,13,1.07371e-12,1.54721e-12,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,
